# VPS Tests with Numpy

These are my first trials to replicate VPS using Numpy.

## Pointshell

We generate 

- A numpy array of shape (10000, 6) which contains random floats in [-1.0, 1.0]; the first 3 columns denote point coordinates, whereas the last 3 columns denote normal vectors. Thus, the 6D elements are points in space each with a normal vector. Additionally, constrain the first 3 floats to be in a unit sphere (i.e., a sphere with radius 1); you can achieve that by projecting the point coordinates onto the unit sphere.
- A numpy array of shape (4,4) containing floats: this array/matrix is in reality a homogeneous transformation matrix H composed by a rotation matrix R (3,3) and a translation vector t (3,1) side by side. Note that the last row of the matrix should be [0.0, 0.0, 0.0, 1.0]. Remember it's a homogeneous transformation matrix; maybe, instead of working with the H (4,4) matrix, we could work with two arrays: R (3,3) and t (3,1). The R part should be orthonormal, i.e., R*transpose(R) = Identity and determinant(R) = 1.0.

In [1]:
import numpy as np

# Function to generate a point on a unit sphere
def generate_on_unit_sphere(n_points):
    vec = np.random.randn(n_points, 3)  # Generate random points in 3D
    vec /= np.linalg.norm(vec, axis=1)[:, np.newaxis]  # Normalize points to lie on the surface of a unit sphere
    return vec

# Generate 10000 points with their normals
points = generate_on_unit_sphere(10000)
normals = np.random.uniform(-1.0, 1.0, (10000, 3))  # Generate random normal vectors
data = np.hstack((points, normals))  # Concatenate points and normals

# Function to generate a random orthonormal matrix, i.e., a rotation matrix
def random_rotation_matrix():
    # Create a random unit quaternion
    q = np.random.randn(4)
    q /= np.linalg.norm(q)

    # Convert the quaternion into a rotation matrix
    q0, q1, q2, q3 = q
    R = np.array([[1 - 2*q2**2 - 2*q3**2, 2*q1*q2 - 2*q3*q0, 2*q1*q3 + 2*q2*q0],
                  [2*q1*q2 + 2*q3*q0, 1 - 2*q1**2 - 2*q3**2, 2*q2*q3 - 2*q1*q0],
                  [2*q1*q3 - 2*q2*q0, 2*q2*q3 + 2*q1*q0, 1 - 2*q1**2 - 2*q2**2]])
    return R

# Generate a random rotation matrix R
R = random_rotation_matrix()

# Generate a random translation vector t
t = np.random.uniform(-1.0, 1.0, (3, 1))

# Construct the homogeneous transformation matrix H
H = np.eye(4)
H[:3, :3] = R
H[:3, 3] = t.ravel()

(data, H)


(array([[ 0.89319603,  0.34896258, -0.28359475,  0.57750775, -0.0621969 ,
          0.11637101],
        [ 0.92236966,  0.35944251, -0.14154606, -0.20340998, -0.17971114,
         -0.19704295],
        [-0.01918837,  0.76569763,  0.64291442, -0.18269594, -0.84653286,
          0.45189174],
        ...,
        [-0.34008641, -0.91017793, -0.23646854, -0.05433948, -0.7709017 ,
          0.79386383],
        [-0.99010847,  0.12693482,  0.05977267,  0.00315547,  0.08403757,
          0.83867625],
        [ 0.89377118,  0.01707179,  0.44819821, -0.35449948, -0.09858001,
         -0.77810782]]),
 array([[-0.17107182, -0.20126035,  0.96448365, -0.93110105],
        [-0.61409151,  0.78729064,  0.05536293, -0.40387504],
        [-0.77047131, -0.58281019, -0.25827551, -0.62324488],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

Now we use R and t to transform data as follows:

- Apply the transformation H to all points in the first 3 columns of data (the points), i.e., new_point = R*point + t
- Apply only the rotation T to the last 3 columns of data (the normals), i.e., new_normal = R*normal

We do it in a function transform_data() which takes as input the data and the H matrix. The function should return the transformed data. However, we should not make any copies! Instead, the input data is modified in place.

We perform 1000 transformations with a unique rando H matrix and measure how long the average transform_data() requires.

In [2]:
import time

# Function to transform data in place according to the provided homogeneous transformation matrix H
def transform_data(data, H, start=None, stop=None):
    R = H[:3, :3]  # Extract the rotation matrix
    t = H[:3, 3]   # Extract the translation vector

    # Determine slice range
    start = start if start is not None else 0
    stop = stop if stop is not None else data.shape[0]

    # Apply transformation to points within the specified slice
    data[start:stop, :3] = np.dot(data[start:stop, :3], R.T) + t

    # Apply rotation to normals within the specified slice
    data[start:stop, 3:] = np.dot(data[start:stop, 3:], R.T)

# Measure performance
start_time = time.time()
for _ in range(1000):
    R = random_rotation_matrix()  # Generate a new random rotation matrix
    t = np.random.uniform(-1.0, 1.0, (3, 1))  # Generate a new random translation vector

    # Construct a new homogeneous transformation matrix H
    H_new = np.eye(4)
    H_new[:3, :3] = R
    H_new[:3, 3] = t.ravel()

    # Transform the data
    # We can transform the complete set of points or a slice start:stop
    start = int(0.0*data.shape[0])
    stop = int(0.99*data.shape[0])
    transform_data(data, H_new, start=start, stop=stop)

end_time = time.time()
average_time = (end_time - start_time) / 1000

average_time # 0.00036285042762756347

0.0003851478099822998

## Voxelmap and Collision

Now we create a class called Environment which will contain a numpy array called map of shape (Nx, Ny, Nz) with random integer values (int16, signed). N values are passed when the class is instantiated. The class should also take the float tuples (xMin, yMin, zMin) and (xMax, yMax, zMax) and save them as the arrays bbox_min and bbox_max of shape (3,).

Additionally, we need a method compute_collision() which takes as input the transformed beforehand data array (10000, 6). Then, the first 3 columns of the data array (points) are checked against the map as follows: 

- The real 3 coordinates of every point p in data are transformed into the discrete map array coordinates using bbox_min and bbox_max: real in data (px, py, pz) -> discrete in map (PX, PY, PZ).
- We create an array of shape (10000, 1) (i.e., in practice the same number of rows as data) which contains 0.0 floats; we call that array penetration and each point as an entry.
- Then, the integer value in the map is observed in the discrete coordinates: map[PX, PY, PZ]; if that value is larger than 0, the corresponding point entry in the penetration array gets assigned the map[PX, PY, PZ] value.
- Finally, the penetration array is multiplied to the last 3 columns of data (normals) one by one and the products are summed. The resulting (1,3) array is the collision force, which is returned by compute_collision().

Do not use any loops, if possible. Instead, use numpy operators only to be as fast as possible.

In [3]:
import numpy as np

class Environment:
    def __init__(self, N, bbox_min, bbox_max):
        self.map = np.random.randint(np.iinfo(np.int16).min, np.iinfo(np.int16).max + 1, N, dtype=np.int16)
        self.bbox_min = np.array(bbox_min)
        self.bbox_max = np.array(bbox_max)

    def compute_collision(self, data, start=None, stop=None):
        # Determine slice range
        start = start if start is not None else 0
        stop = stop if stop is not None else data.shape[0]

        # Scale factors for converting real coordinates to discrete map indices
        scale_factors = (self.map.shape - np.array([1, 1, 1])) / (self.bbox_max - self.bbox_min)

        # Convert real coordinates in the specified slice of data to discrete coordinates in the map
        # TODO: store distance from voxel center to point and project it in normal to correct map_value
        discrete_coords = ((data[start:stop, :3] - self.bbox_min) * scale_factors).astype(int)

        # Clamp coordinates to map bounds
        discrete_coords = np.clip(discrete_coords, [0, 0, 0], np.array(self.map.shape) - 1)

        # Fetch map values at discrete coordinates without using a loop
        map_values = self.map[discrete_coords[:, 0], discrete_coords[:, 1], discrete_coords[:, 2]]

        # Update penetration values based on map values
        # TODO: get max/min penetration/distance
        # TODO: make penetration/distance array a property that can be accessed
        penetration = np.where(map_values > 0, map_values, 0).reshape(-1, 1)

        # Compute collision force for the specified slice
        # TODO: compute torque approximations
        collision_force = np.sum(penetration * data[start:stop, 3:], axis=0)

        return collision_force.reshape((1, 3))

# Example of class instantiation and method usage
N = (100, 100, 100)  # Map dimensions
bbox_min = (-5.0, -5.0, -5.0)  # Minimum bounds
bbox_max = (5.0, 5.0, 5.0)  # Maximum bounds

env = Environment(N, bbox_min, bbox_max)

# For demonstration purposes, let's create a sample data array similar to the previously transformed data
data_sample = np.random.uniform(-1.0, 1.0, (10000, 6))
data_sample[:, :3] /= np.linalg.norm(data_sample[:, :3], axis=1)[:, np.newaxis]  # Normalize first 3 columns to unit vectors

# Compute the collision force
collision_force = env.compute_collision(data_sample)
collision_force

array([[ 387627.86741652, -461220.64411773,  643402.43760574]])

Now we run compute_collision(data_sample) 1000 times and measure the average time it takes.

In [4]:
import time

# Measure performance of the optimized compute_collision method again, now with time module imported
start_time_optimized = time.time()

for _ in range(1000):
    start = int(0.0*data_sample.shape[0])
    stop = int(0.99*data_sample.shape[0])
    env.compute_collision(data_sample, start=start, stop=stop)

end_time_optimized = time.time()
average_time_optimized = (end_time_optimized - start_time_optimized) / 1000

average_time_optimized


0.0006852850914001465